In [ ]:
import os
import sys
import fnmatch
import zipfile
import xmltodict
import numpy as np
import pandas as pd
import json
import gzip
import pickle
import csv
import scipy.sparse

In [ ]:
#df = pd.read_hdf('papers.h5','table')
df = pd.read_pickle('papers.pkl.bz2')


In [ ]:
import bz2
main_log = pickle.load(bz2.BZ2File('../CSrankings/main_log.pkl.bz2','rb'))

In [ ]:
unique_names = pickle.load(open('big_names.pkl','rb'))
unique_confs = pickle.load(open('confs.pkl','rb'))

In [ ]:
#unique_names = set(sum([list(_) for _ in df.name],[]))

In [ ]:
name_set = {}
for names in df.name:
    nv = {k: 1 for k in names}
    name_set.update(nv)
unique_names = list(name_set)

In [ ]:
for row in df.itertuples():
    for a in row[3]:
        if a == b'Angela Dai':
            print(row)


In [ ]:
for paper in main_log:
    for a in paper['name']:
        if a == b'Angela Dai':
            print(paper)


In [ ]:
r1_confs = pickle.load(open('r1_confs.pkl','rb'))
r1_confs_dict = {_:1 for _ in r1_confs}

In [ ]:
# from the big paper thing
papers = pd.read_hdf('papers.h5','table')
unique_names = pickle.load(open('big_names.pkl','rb'))
unique_confs = pickle.load(open('confs.pkl','rb'))

faculty_affil = pd.read_csv('faculty-affiliations.csv')
ranks = pd.read_csv('ranks.csv')
def csv2dict_str_str(fname):
    with open(fname, mode='r') as infile:
        rdr = csv.reader(infile)
        d = {rows[0].strip(): rows[1].strip() for rows in rdr}
    return d
aliasdict = csv2dict_str_str('dblp-aliases.csv')
conf_idx = pickle.load(open('conf_idx.pkl','rb'))
name_idx = pickle.load(open('name_idx.pkl','rb'))

In [ ]:
areadict = {
    'icse' : ['ICSE', 'ICSE (1)'],
    'fse'  : ['SIGSOFT FSE', 'ESEC/SIGSOFT FSE'],
    'usenixatc' : ['USENIX Annual Technical Conference', 'USENIX Annual Technical Conference, General Track'], # next tier
    'imc': ['IMC', 'Internet Measurement Conference'],
    'sigmetrics': ['SIGMETRICS', 'SIGMETRICS/Performance', 'POMACS'],
    'mobicom' : ['MobiCom', 'MOBICOM'],
    'rtas' : ['RTAS', 'IEEE Real-Time and Embedded Technology and Applications Symposium'],
    'ccs': ['CCS', 'ACM Conference on Computer and Communications Security'],
    'oakland' : ['IEEE Symposium on Security and Privacy'],
    'usenixsec' : ['USENIX Security Symposium', 'USENIX Security'],
    'pets' : ['PoPETs', 'Privacy Enhancing Technologies'],
    'cav': ['CAV', 'CAV (1)', 'CAV (2)'],
    'lics' : ['LICS', 'CSL-LICS'],
    'nips': ['NIPS', 'NeurIPS'],
    'icml': ['ICML', 'ICML (1)', 'ICML (2)', 'ICML (3)'],
    'aaai': ['AAAI', 'AAAI/IAAI'],
    'ubicomp' : ['UbiComp', 'Ubicomp', 'IMWUT', 'Pervasive'],
    'emnlp': ['EMNLP', 'EMNLP-CoNLL', 'HLT/EMNLP'],
    'acl' : ['ACL', 'ACL (1)', 'ACL (2)', 'ACL/IJCNLP', 'COLING-ACL'],
    'naacl' : ['NAACL', 'HLT-NAACL', 'NAACL-HLT'],
    'cvpr': ['CVPR', 'CVPR (1)', 'CVPR (2)'],
    'eccv': ['ECCV', 'ECCV (1)', 'ECCV (2)', 'ECCV (3)', 'ECCV (4)', 'ECCV (5)', 'ECCV (6)', 'ECCV (7)', 'ECCV (8)', 'ECCV (9)', 'ECCV (10)', 'ECCV (11)', 'ECCV (12)', 'ECCV (13)', 'ECCV (14)', 'ECCV (15)', 'ECCV (16)'],
    'icra': ['ICRA', 'ICRA (1)', 'ICRA (2)'],
    'rss': ['Robotics: Science and Systems'],
    'crypto': ['CRYPTO', 'CRYPTO (1)', 'CRYPTO (2)', 'CRYPTO (3)'],
    'eurocrypt': ['EUROCRYPT', 'EUROCRYPT (1)', 'EUROCRYPT (2)', 'EUROCRYPT (3)'],
}
inverse_area_dict = {}
for k,v in areadict.items():
    n = len(v)
    for i in range(1,n):
        inverse_area_dict[v[i]] = v[0]
for k,v in inverse_area_dict.items():
    if k in conf_idx and v in conf_idx:
        conf_idx[k] = conf_idx[v]

In [ ]:
# munge the years
min_year = papers.year.min()
max_year = papers.year.max()
span_years = max_year - min_year +1

print(span_years,min_year,max_year,len(name_idx))
ranks[ranks.index < 16]

In [ ]:
import scipy.stats
import matplotlib.pyplot as plt
sigma = 3
weights = []
for i in range(span_years):
    a = np.array([scipy.stats.norm.pdf( (j-i)/sigma) for j in range(span_years)])
    a[a < 0.05] = 0
    weights.append(a/np.linalg.norm(a))
_ = plt.plot(np.arange(span_years)+min_year,weights[2000-min_year])
plt.grid(True)

In [ ]:
import itertools
#pairs_of_years = itertools.product(range(span_years),range(span_years))

wdict = {}
for i,j,k in itertools.product(range(unique_confs.shape[0]),range(span_years),range(span_years)):
    wdict[i*span_years+j,i*span_years+k] = weights[j][k]
wsa = scipy.sparse.dok_matrix((span_years*unique_confs.shape[0],span_years*unique_confs.shape[0]))
wsa._update(wdict)

In [ ]:
import scipy.sparse
X = scipy.sparse.dok_matrix((len(unique_names),span_years*unique_confs.shape[0]))
xdict = {}

auth_years = np.ones((len(unique_names),2)) * np.array([3000,1000]) 
#y = np.zeros(len(unique_names))#scipy.sparse.dok_matrix((1,len(unique_names)))
for row in df.itertuples():
    paper_year = row[10]

    conf = row[2]
    n = row[4]
    authors = row[3]
    j = span_years*conf_idx[conf] + paper_year-min_year
    for a in authors:
        i = name_idx[a]
        xdict[(i,j)] = 1/n + xdict.get((i,j),0)
        auth_years[i,0] = min(auth_years[i,0],paper_year)
        auth_years[i,1] = max(auth_years[i,1],paper_year)
X._update(xdict)


In [ ]:
X = scipy.sparse.csr_matrix(X)
wsa = scipy.sparse.csr_matrix(wsa)
X = X @ wsa

In [ ]:
#settings = [-4.228109135925832,'modified_huber',31,	0.919015912687292,0.7758551488895429] # no errors but little acc
settings =[-6.013005893126464,	'modified_huber',	30,	0.9485296659531548,	0.683735528500138] # errors but acc


In [ ]:
y = np.zeros(len(unique_names))
for i in range(5): #40?
    uni_name = ranks.iloc[i]['uni']
    uni_faculty = faculty_affil[faculty_affil.affiliation == uni_name]
    uni_names = np.array(uni_faculty.name)
    for name in set([aliasdict.get(n, n) for n in uni_names]):
        if name in name_idx:
            y[name_idx[name]] = 1
        else:
            pass
            #print(name)

In [ ]:
nonarxiv = np.ones(span_years*len(unique_confs))
nonarxiv[span_years*conf_idx['CoRR']:span_years*(conf_idx['CoRR']+1)] = 0
r1_confs = np.ones(span_years*len(unique_confs))

for k,v in r1_confs_dict.items():
    pass
    #r1_confs[span_years*conf_idx[k]:span_years*(conf_idx[k]+1)] = 1

skipped_conf = scipy.sparse.diags(nonarxiv * r1_confs)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVR

X = scipy.sparse.csr_matrix(X)
clf = SGDClassifier(settings[1],average=False,verbose=1,warm_start=True,tol=1e-5,max_iter=1,alpha=1e-3)
tmp = X@ skipped_conf
clf.fit(tmp,y)
for i in range(35):
    minv = clf.coef_[clf.coef_ > 0].min()
    maxv = clf.coef_[clf.coef_ > 0].max()
    #clf.coef_ = np.maximum(minv,clf.coef_)
    clf = clf.partial_fit(tmp,y)
    print(minv,maxv)
minv = clf.coef_[clf.coef_ > 0].min()
#clf.coef_ = np.maximum(minv,clf.coef_)
tmp = None

In [ ]:
conf_results = np.squeeze(clf.coef_)
from scipy.ndimage.filters import gaussian_filter1d

conf_ord = np.argsort(conf_results)
conf_choice = ['SIGGRAPH','HRI','ECCV','Comput. Graph. Forum','Shape Modeling International','Symposium on Geometry Processing','Computer Aided Geometric Design','I. J. Robotics Res.','CVPR','International Journal of Computer Vision','Robotics: Science and Systems','ICRA','WACV','ICML','AISTATS','CoRR','SIGGRAPH Asia','ECCV','ICCV','ISER','Humanoids','3DV','IROS','CoRL','Canadian Conference on AI','ACCV','Graphics Interface','CRV','BMVC']
ri_confs = np.zeros(len(unique_confs)*span_years)
print(clf.intercept_)
ms = conf_results.mean()
ss = conf_results.std()
seen = {}
for i in range(len(unique_confs)*span_years):
    idx = conf_ord[-(i+1)]
    conf_name = unique_confs[idx//span_years]
    conf_score = conf_results[idx]
    if conf_name in conf_choice:
        ri_confs[idx] = 1
    if conf_name in conf_choice and conf_name not in seen:
        print('{:20s}{}\t{:.1f}'.format(conf_name[:20],str(min_year + (idx % span_years)),(conf_score-ms)/ss))
        seen[conf_name] =1
ri_confs.shape,ri_confs.sum(),X.shape

conf_choice2 = ['SIGGRAPH','AAAI','NIPS','CVPR','ICRA','ICML','ICCV',
               'International Journal of Computer Vision','Robotics: Science and Systems']
conf_choice3 = []
vs = conf_results.std()
for conf in conf_choice2:
    idx = conf_idx[conf]
    s = max(conf_results[span_years*idx:span_years*(idx+1)])
    conf_choice3.append((s,conf))
plt.figure(figsize=(12,8))
for s,conf in sorted(conf_choice3,reverse=True):
    idx = conf_idx[conf]
    _ = plt.plot(np.arange(min_year,max_year+1)[:-4],gaussian_filter1d(conf_results[span_years*idx:span_years*(idx+1)]/vs,sigma=1)[:-4],label=conf)
plt.grid()
plt.xlabel('year')
plt.ylabel('value')
plt.legend()
plt.savefig('faculty-fixed.pdf')
plt.figure()
conf_choice2 = ['CVPR','ECCV','BMVC','CRV','ICCV']
conf_choice3 = []
vs = conf_results.std()
for conf in conf_choice2:
    idx = conf_idx[conf]
    s = max(conf_results[span_years*idx:span_years*(idx+1)])
    conf_choice3.append((s,conf))
plt.figure(figsize=(12,8))
for s,conf in sorted(conf_choice3,reverse=True):
    idx = conf_idx[conf]
    _ = plt.plot(np.arange(min_year,max_year+1)[:-4],gaussian_filter1d(conf_results[span_years*idx:span_years*(idx+1)]/vs,sigma=1)[:-4],label=conf)
plt.grid()
plt.xlabel('year')
plt.ylabel('value')
plt.legend()
plt.figure()
conf_choice2 = ['AAAI','IJCAI','UAI','AAMAS']
conf_choice3 = []
vs = conf_results.std()
for conf in conf_choice2:
    idx = conf_idx[conf]
    s = max(conf_results[span_years*idx:span_years*(idx+1)])
    conf_choice3.append((s,conf))
plt.figure(figsize=(12,8))
for s,conf in sorted(conf_choice3,reverse=True):
    idx = conf_idx[conf]
    _ = plt.plot(np.arange(min_year,max_year+1)[:-4],gaussian_filter1d(conf_results[span_years*idx:span_years*(idx+1)]/vs,sigma=1)[:-4],label=conf)
plt.grid()
plt.xlabel('year')
plt.ylabel('value')
plt.legend()

In [ ]:
top_k = 50
i = -1
j = 0
seen = {}
while j < top_k:
    i += 1
    idx = conf_ord[-(i+1)]
    conf_name = unique_confs[idx//span_years]
    if conf_name in seen:
        continue
    j+=1
    conf_score = conf_results[idx]
    seen[conf_name] = 1
    print('{:20s}\t{}\t\t{:.3f}\t{:.2f}'.format(conf_name[:18],min_year + (idx % span_years),100*conf_score,(conf_score-ms)/ss))

In [ ]:
#_ = hist(clf.coef_,70)
pickle.dump(conf_results,open('faculty_indep-fixed_16.pkl','wb'))


In [ ]:
if Xauth is None or (Xauth.shape[1] != span_years*unique_confs.shape[0]):  
    Xauth = scipy.sparse.dok_matrix((len(unique_names),span_years*unique_confs.shape[0]))
    xdict = {}
    auth_years = np.ones((len(unique_names),2)) * np.array([3000,1000]) 
    for row in papers.itertuples():
        paper_year = row[10]
        #if row['year'] < 2005:
        #    continue
        #print(row)
        #if row['conf'] == 'CoRR':
        #    continue
        conf = row[2]
        n = row[4]
        authors = row[3]
        j = span_years*conf_idx[conf] + (paper_year-min_year)
        for a in authors:
            i = name_idx[a]
            xdict[(i,j)] = 1/n + xdict.get((i,j),0)
            auth_years[i,0] = min(auth_years[i,0],paper_year)
            auth_years[i,1] = max(auth_years[i,1],paper_year)
    Xauth._update(xdict)

In [ ]:
scores = clf.predict(Xauth) - np.squeeze(clf.intercept_)
years_working = (1+auth_years[:,1]-auth_years[:,0])
value_scores = scores
norm_scores = (value_scores)/years_working
ri_filter_mat = scipy.sparse.diags(ri_confs)
ri_scores = clf.predict(Xauth.dot(ri_filter_mat))-np.squeeze(clf.intercept_)
ri_norm_scores = ri_scores/years_working

In [ ]:
prev_cand = ['Pulkit Agrawal',
 'Joydeep Biswas',
 'Katherine L. Bouman',
 'David Braun',
 'Jia Deng',
 'Naomi T. Fitter',
 'David F. Fouhey',
 'Saurabh Gupta',
 'Judy Hoffman',
 'Hanbyul Joo',
 'Honglak Lee',
 'Changliu Liu',
 'Petter Nilsson',
 "Matthew O'Toole",
 'Alessandro Roncone',
 'Alanson P. Sample',
 'Manolis Savva',
 'Adriana Schulz',
 'Amy Tabb',
 'Fatma Zeynep Temel',
 'Long Wang',
 'Cathy Wu',
 'Ling-Qi Yan']
print('{:20s}\t{:4s}\t{:4s}\t{:4s}\t{}'.format('name','rate','total','ri','years'))
for ns, name in sorted([(value_scores[name_idx[ni]],ni) for ni in prev_cand],reverse=True):
    ni = name_idx[name]
    print('{:20s}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.0f}'.format(name,100*norm_scores[ni],100*value_scores[ni],100*ri_scores[ni],years_working[ni]))
print('')
curious_names = ['Xiaolong Wang 0004','Judy Hoffman','Paris Siminelakis',
                 'Nicholas Rhinehart',
                 'Humphrey Hu',
                 'David F. Fouhey',
                 'Lerrel Pinto',
                 'Justin Johnson',
                 'Amir Roshan Zamir',
                 'Brian Okorn','David Held']
print('{:20s}\t{:4s}\t{:4s}\t{:4s}\t{}'.format('name','rate','total','ri','years'))
for _,name in sorted([(value_scores[name_idx[_]],_) for _ in curious_names],reverse=True):
    ni = name_idx[name]
    print('{:20s}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.0f}'.format(name,100*norm_scores[ni],100*value_scores[ni],100*ri_scores[ni],years_working[ni]))

In [ ]:
print('\n best overall \n')
cmu_scores = []

best_scores = np.argsort(value_scores)[::-1]
#print(best_scores.shape,unique_names[best_scores[0]])
fa_list = list(faculty_affil.name)
fa_a_list = list(faculty_affil.affiliation)
uni_names = [unique_names[i] for i in best_scores[:38000]]
for name in set([aliasdict.get(n, n) for n in uni_names]):
    if name in name_idx:
        uni = 'unknown'
        if name in fa_list:
            uni = fa_a_list[fa_list.index(name)]
        if name not in []:#['Jacob Walker','Justin Johnson','Pieter Abbeel','Martial Hebert','Jessica K. Hodgins','Abhinav Gupta','Christopher G. Atkeson','Tom M. Mitchell','Matthew T. Mason']:
            if years_working[name_idx[name]] < 3:
                continue
            if years_working[name_idx[name]] > 8:
                continue
            if ri_scores[name_idx[name]] < 0.008:
                continue
            if auth_years[name_idx[name],1] < 2017:
                continue
        #if (np.array(X[name_idx[name],:].todense()) * ri_confs).sum() == 0:
        #    continue
        #print(name,auth_years[name_idx[name]])
        score = norm_scores[name_idx[name]]
        ri_vscore = ri_norm_scores[name_idx[name]]
        vscore = value_scores[name_idx[name]]
        cmu_scores.append((vscore,ri_scores[name_idx[name]],score,uni,name,auth_years[name_idx[name]],ri_vscore))
    else:
        pass
        #print(name)
        ri_norm_scores
print('{:22s}\t{:15s}\t{:5s}\t{:3s}\t{:4s}\t{:4s}\t{} {}'.format('name','uni','rate','RI-t','total','RI-r','start','end'))
for vs,ris,s,u,p,yrs,rir in sorted(cmu_scores,reverse=True):
    print('{:22s}\t{:15s}\t{:.3f}\t{:.1f}\t{:.2f}\t{:.2f}\t{} {}'.format(p[:22],u[:15],s*100,ris*100,vs*100,rir*100,int(yrs[0]),int(yrs[1])))


In [ ]:
uni_faculty = faculty_affil[faculty_affil.affiliation == 'Carnegie Mellon University'] #Carnegie Mellon University
uni_names = np.array(uni_faculty.name)
uni_names = list(uni_names) + ['Nicholas Rhinehart','Jacob Walker','Lerrel Pinto','Brian Okorn','Leonid Keselman','Siddharth Ancha','Humphrey Hu']
cmu_scores = []
#uni_names = [unique_names[i] for i in (np.argsort(scores)[::-1])[:150]]
for name in set([aliasdict.get(n, n) for n in uni_names]):
    if name in name_idx:
        #if ri_scores[name_idx[name]] < 2.5:
        #    continue
        score = scores[name_idx[name]]
        cmu_scores.append((score,name))
    else:
        pass
        #print(name)
for s,p in sorted(cmu_scores,reverse=True):
    print('{:30s}\t\t{:.3f}'.format(p,s*100))


In [ ]:
pickle.dump(Xauth,open('xauth.pkl','wb'))


In [ ]:
du -h *.pkl

In [ ]:
import gc
gc.collect()